In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="results", extension="png", remove_last=False)
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,Architecture
8,model_4_9_8,0.999873,0.784304,0.999353,0.999664,0.999432,0.000535,1.442359,0.001523,0.000160,0.000842,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
12,model_4_9_7,0.999866,0.784143,0.999437,0.999693,0.999504,0.000563,1.443437,0.001325,0.000146,0.000736,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
18,model_4_9_6,0.999856,0.783963,0.999524,0.999722,0.999577,0.000605,1.444642,0.001121,0.000132,0.000627,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
19,model_4_9_5,0.999842,0.783761,0.999611,0.999751,0.999651,0.000665,1.445988,0.000915,0.000119,0.000517,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
20,model_4_9_4,0.999822,0.783536,0.999698,0.999778,0.999725,0.000747,1.447494,0.000710,0.000106,0.000408,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
21,model_4_9_3,0.999795,0.783284,0.999781,0.999803,0.999795,0.000859,1.449179,0.000514,0.000094,0.000304,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
22,model_4_9_2,0.999760,0.783002,0.999858,0.999823,0.999859,0.001007,1.451067,0.000335,0.000084,0.000210,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
26,model_4_9_1,0.999713,0.782686,0.999922,0.999835,0.999912,0.001202,1.453182,0.000183,0.000078,0.000131,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
33,model_2_8_8,0.999696,0.762694,0.999417,0.999749,0.999662,0.001275,1.586865,0.002212,0.001731,0.001972,"Hidden Size=[30, 16], regularizer=0.02, learni..."
37,model_2_8_7,0.999679,0.762509,0.999491,0.999782,0.999705,0.001345,1.588106,0.001930,0.001509,0.001719,"Hidden Size=[30, 16], regularizer=0.02, learni..."
